<a href="https://colab.research.google.com/github/vicmcl/ml-laptime/blob/main/ml_laptime/notebook/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from pathlib import Path
import os
import glob

import pandas as pd
import pickle

In [2]:
# Mount Drive
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive')
DRIVE_PATH = Path("/content/drive/MyDrive/")
ML_PATH = DRIVE_PATH / "Projects" / "laptime-simulation"

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [3]:
data_path = glob.glob(str(ML_PATH / "*.pkl"))

In [8]:
pip install -q fastf1

In [5]:
with open(data_path[0], "rb") as f:
    data = pickle.load(f)

In [6]:
session = data['Abu Dhabi Grand Prix']

In [129]:
columns_laps = [
    'Time', 'LapNumber', 'Stint', 'Sector1Time', 'Sector2Time',
    'Sector3Time', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'Compound',
    'TyreLife', 'Position'
]

In [130]:
laps_data = session['laps'][columns_laps]

In [131]:
laps_data = laps_data.reset_index().drop('index', axis=1)

In [132]:
laps_data

,Time,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,SpeedFL,SpeedST,Compound,TyreLife,Position
0,0 days 01:03:10.370000,1.0,1.0,NaT,0 days 00:00:38.016000,0 days 00:00:17.233000,296.0,249.0,295.0,214.0,MEDIUM,1.0,1.0
1,0 days 01:04:25.050000,2.0,1.0,0 days 00:00:19.302000,0 days 00:00:38.122000,0 days 00:00:17.256000,297.0,246.0,296.0,280.0,MEDIUM,2.0,1.0
2,0 days 01:05:39.521000,3.0,1.0,0 days 00:00:19.090000,0 days 00:00:38.134000,0 days 00:00:17.247000,301.0,250.0,298.0,285.0,MEDIUM,3.0,1.0
3,0 days 01:06:54.234000,4.0,1.0,0 days 00:00:19.121000,0 days 00:00:38.283000,0 days 00:00:17.309000,NaN,241.0,298.0,284.0,MEDIUM,4.0,1.0
4,0 days 01:08:08.894000,5.0,1.0,0 days 00:00:19.185000,0 days 00:00:38.312000,0 days 00:00:17.163000,303.0,244.0,299.0,287.0,MEDIUM,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0 days 02:29:27.171000,67.0,5.0,0 days 00:00:18.978000,0 days 00:00:37.897000,0 days 00:00:17.083000,309.0,253.0,303.0,304.0,HARD,27.0,2.0
67,0 days 02:30:41.596000,68.0,5.0,0 days 00:00:18.985000,0 days 00:00:38.207000,0 days 00:00:17.233000,308.0,251.0,296.0,301.0,HARD,28.0,2.0
68,0 days 02:31:55.985000,69.0,5.0,0 days 00:00:19.205000,0 days 00:00:37.997000,0 days 00:00:17.187000,303.0,251.0,303.0,291.0,HARD,29.0,2.0
69,0 days 02:33:10.392000,70.0,5.0,0 days 00:00:19.089000,0 days 00:00:38.098000,0 days 00:00:17.220000,307.0,250.0,302.0,297.0,HARD,30.0,2.0


In [36]:
session['telemetry'].columns

Index(['Date', 'SessionTime', 'DriverAhead', 'DistanceToDriverAhead', 'Time',
       'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
       'Distance', 'RelativeDistance', 'Status', 'X', 'Y', 'Z'],
      dtype='object')

In [144]:
telemetry_columns = ['SessionTime', 'RPM', 'nGear', 'Throttle']

In [145]:
telemetry_data = session['telemetry'][telemetry_columns]

In [146]:
telemetry_data.head()

,SessionTime,RPM,nGear,Throttle
2,0 days 01:01:51.781000,10936,1,15
3,0 days 01:01:51.890000,10973,1,15
4,0 days 01:01:51.975000,10763,1,15
5,0 days 01:01:52.130000,10553,1,15
6,0 days 01:01:52.195000,9713,1,15


In [147]:
telemetry_with_laps = pd.merge_asof(
    telemetry_data,
    laps_data[['Time', 'LapNumber']],
    left_on='SessionTime',
    right_on='Time',
    direction='forward'
).drop('Time', axis=1)

In [148]:
telemetry_with_laps.head()

,SessionTime,RPM,nGear,Throttle,LapNumber
0,0 days 01:01:51.781000,10936,1,15,1.0
1,0 days 01:01:51.890000,10973,1,15,1.0
2,0 days 01:01:51.975000,10763,1,15,1.0
3,0 days 01:01:52.130000,10553,1,15,1.0
4,0 days 01:01:52.195000,9713,1,15,1.0


In [149]:
gear_shifts = telemetry_with_laps.groupby('LapNumber').apply(
    lambda lap: (lap['nGear'].diff().abs() != 0).sum()
)

In [118]:
gear_shifts

LapNumber
1.0     40
2.0     35
3.0     35
4.0     35
5.0     34
        ..
67.0    34
68.0    36
69.0    36
70.0    37
71.0    36
Length: 71, dtype: int64

In [150]:
telemetry_avg = telemetry_with_laps.drop(['nGear', 'SessionTime'], axis=1).groupby('LapNumber').mean()

In [151]:
telemetry_avg = pd.concat([telemetry_avg, gear_shifts], axis=1).rename(columns={0: 'GearShifts'})

In [152]:
telemetry_avg

,RPM,Throttle,GearShifts
LapNumber,,,
1.0,10313.163504,65.327007,40
2.0,10123.721538,64.527692,35
3.0,10128.352308,64.089231,35
4.0,10094.453005,62.656394,35
5.0,10068.790447,63.229584,34
...,...,...,...
67.0,10150.404651,61.269767,34
68.0,10147.502311,60.810478,36
69.0,10086.973725,61.536321,36


In [153]:
merged_data = pd.merge_asof(laps_data, session['weather'], on='Time', direction='nearest').set_index('LapNumber').drop('Time', axis=1)
merged_data = pd.concat([merged_data, telemetry_avg], axis=1).reset_index()

In [154]:
merged_data

,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,SpeedFL,SpeedST,Compound,...,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,RPM,Throttle,GearShifts
0,1.0,1.0,NaT,0 days 00:00:38.016000,0 days 00:00:17.233000,296.0,249.0,295.0,214.0,MEDIUM,...,22.8,61.9,927.0,False,52.1,164,0.5,10313.163504,65.327007,40
1,2.0,1.0,0 days 00:00:19.302000,0 days 00:00:38.122000,0 days 00:00:17.256000,297.0,246.0,296.0,280.0,MEDIUM,...,23.0,62.5,926.9,False,51.8,178,0.4,10123.721538,64.527692,35
2,3.0,1.0,0 days 00:00:19.090000,0 days 00:00:38.134000,0 days 00:00:17.247000,301.0,250.0,298.0,285.0,MEDIUM,...,23.1,61.6,927.0,False,51.7,193,0.8,10128.352308,64.089231,35
3,4.0,1.0,0 days 00:00:19.121000,0 days 00:00:38.283000,0 days 00:00:17.309000,NaN,241.0,298.0,284.0,MEDIUM,...,23.0,62.9,927.0,False,51.9,339,0.5,10094.453005,62.656394,35
4,5.0,1.0,0 days 00:00:19.185000,0 days 00:00:38.312000,0 days 00:00:17.163000,303.0,244.0,299.0,287.0,MEDIUM,...,23.0,61.9,926.8,False,51.9,156,0.5,10068.790447,63.229584,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,67.0,5.0,0 days 00:00:18.978000,0 days 00:00:37.897000,0 days 00:00:17.083000,309.0,253.0,303.0,304.0,HARD,...,22.8,63.2,926.7,False,46.3,169,0.4,10150.404651,61.269767,34
67,68.0,5.0,0 days 00:00:18.985000,0 days 00:00:38.207000,0 days 00:00:17.233000,308.0,251.0,296.0,301.0,HARD,...,22.8,63.0,926.7,False,46.1,178,0.3,10147.502311,60.810478,36
68,69.0,5.0,0 days 00:00:19.205000,0 days 00:00:37.997000,0 days 00:00:17.187000,303.0,251.0,303.0,291.0,HARD,...,22.7,62.8,926.7,False,45.5,197,0.4,10086.973725,61.536321,36
69,70.0,5.0,0 days 00:00:19.089000,0 days 00:00:38.098000,0 days 00:00:17.220000,307.0,250.0,302.0,297.0,HARD,...,22.7,63.2,926.6,False,44.6,184,0.5,10120.798151,60.864407,37
